In [1]:
import glob
import numpy as np
import pickle
import os
from collections import Counter
from utils.spoofeval import spoof_eval

In [2]:
ppath='/Work29/wwm1995/SMIIP/wxm_spoof2021/best_score/'
score_path=os.path.join(ppath,'*/score_path_*.log')
score_path=glob.glob(score_path)

In [3]:
print(score_path[0])

/Work29/wwm1995/SMIIP/wxm_spoof2021/best_score/19LA_flac_Aug1_80FBANK_ResNet18_ASP_fft1024len64hop8/score_path_10.log


In [4]:
def zscore(x, axis = None):
    xmean = x.mean(axis=axis, keepdims=True)
    xstd  = np.std(x, axis=axis, keepdims=True)
    zscore = (x-xmean)/xstd
    return zscore

In [5]:
a=np.random.rand(2,4)
print(a,'\n')

print(a.mean(axis=1))
print(np.std(a,axis=1))

zscore(a,1)

[[0.68211919 0.03763295 0.24360145 0.18401394]
 [0.53460389 0.58767782 0.284668   0.6791908 ]] 

[0.28684188 0.52153513]
[0.24020437 0.1462079 ]


array([[ 1.64558748, -1.03748707, -0.18001517, -0.42808523],
       [ 0.08938479,  0.45238796, -1.62007069,  1.07829794]])

### Input Your model parameters

In [6]:
seed=[1,10,100]
dataset=['19LA','21LA']
model=['ResNet18','LightCNN_lstm']
upsample=['flac','BE_TDclean','BE_TDnoise']
aug=[0,1]

In [7]:
score_path_1=f'/Work29/wwm1995/SMIIP/wxm_spoof2021/best_score/{dataset[1]}_{upsample[2]}_Aug{aug[1]}_80FBANK_{model[0]}_ASP_fft1024len64hop8/score_path_{seed[2]}.log'
print(score_path_1)
score_path_2=f'/Work29/wwm1995/SMIIP/wxm_spoof2021/best_score/{dataset[1]}_{upsample[2]}_Aug{aug[1]}_80FBANK_{model[1]}_ASP_fft1024len64hop8/score_path_{seed[2]}.log'
print(score_path_2)

with open(str(score_path_1), 'r') as sp:
    score_file=sp.readline()
    score_file1 = score_file.rstrip() 


with open(str(score_path_2), 'r') as sp:
    score_file=sp.readline()
    score_file2 = score_file.rstrip() 


/Work29/wwm1995/SMIIP/wxm_spoof2021/best_score/21LA_BE_TDnoise_Aug1_80FBANK_ResNet18_ASP_fft1024len64hop8/score_path_100.log
/Work29/wwm1995/SMIIP/wxm_spoof2021/best_score/21LA_BE_TDnoise_Aug1_80FBANK_LightCNN_lstm_ASP_fft1024len64hop8/score_path_100.log


### FUSION 实现方法一：使用collections.Counter方法直接同名元素相加，创建字典

In [8]:
with open(score_file1, 'rb') as f:
    score_1=pickle.load(f)
with open(score_file2, 'rb') as f:
    score_2=pickle.load(f)

score_tmp=np.array([v1 for _, v1 in score_1.items()])
score_tmp=zscore(score_tmp,axis=1)
score_1={k1:v for (k1,_),v in zip(score_1.items(),score_tmp)}

score_tmp=np.array([v2 for _, v2 in score_2.items()])
score_tmp=zscore(score_tmp,axis=1)
score_2={k2:v for (k2,_),v in zip(score_2.items(),score_tmp)}

#fusion weight
# a=0.2
# score_1 = {k1:(1-a)*v1 for k1, v1 in score_1.items()}
# score_2 = {k2:a*v2 for k2, v2 in score_2.items()}

dicts=[score_1,score_2]
c = Counter()
for d in dicts:
    # c.subtract(d)
    c.update(d)

In [9]:
print('PRAINTING single system 1 SCORES BASED RESULTS!!!\n')
val_data_name='asvspoof21LA/eval'
scores_dic=score_1


ACC, EER, threshold= spoof_eval('data/%s/utt2label' % val_data_name , scores_dic, pos_label=0 )
print('ACC:%f EER:%f Threshold:%f\n'%(ACC,EER,threshold))

ACC, EER, threshold= spoof_eval('data/%s/utt2label' % val_data_name , scores_dic, pos_label=1 )
print('ACC:%f EER:%f Threshold:%f\n'%(ACC,EER,threshold))
print('PRAINTING single system 2 SCORES BASED RESULTS!!!\n')
scores_dic=score_2


ACC, EER, threshold= spoof_eval('data/%s/utt2label' % val_data_name , scores_dic, pos_label=0 )
print('ACC:%f EER:%f Threshold:%f\n'%(ACC,EER,threshold))

ACC, EER, threshold= spoof_eval('data/%s/utt2label' % val_data_name , scores_dic, pos_label=1 )
print('ACC:%f EER:%f Threshold:%f\n'%(ACC,EER,threshold))
print('PRAINTING fusion SCORES BASED RESULTS!!!\n')

scores_dic=dict(c)

ACC, EER, threshold= spoof_eval('data/%s/utt2label' % val_data_name , scores_dic, pos_label=0 )
print('ACC:%f EER:%f Threshold:%f\n'%(ACC,EER,threshold))

ACC, EER, threshold= spoof_eval('data/%s/utt2label' % val_data_name , scores_dic, pos_label=1 )
print('ACC:%f EER:%f Threshold:%f\n'%(ACC,EER,threshold))

PRAINTING single system 1 SCORES BASED RESULTS!!!

{'bonafide': 0, 'spoof': 1}
ACC:96.464340 EER:2.351208 Threshold:1.000000

{'bonafide': 0, 'spoof': 1}
ACC:96.464340 EER:2.350083 Threshold:-1.000000

PRAINTING single system 2 SCORES BASED RESULTS!!!

{'bonafide': 0, 'spoof': 1}
ACC:97.087922 EER:2.673734 Threshold:1.000000

{'bonafide': 0, 'spoof': 1}
ACC:97.087922 EER:2.672983 Threshold:-1.000000

PRAINTING fusion SCORES BASED RESULTS!!!

{'bonafide': 0, 'spoof': 1}
ACC:95.939288 EER:2.525657 Threshold:0.000000

{'bonafide': 0, 'spoof': 1}
ACC:95.939288 EER:2.504659 Threshold:0.000000

